<a href="https://colab.research.google.com/github/mongrelDS/MsDS-CoLab/blob/main/shopping_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# @title Initialize
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from datetime import datetime, timedelta
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import urllib.parse
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

import os
import io
import re

# Initialize Google Drive API client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [58]:
# @title Load Shoppng 5 Column File
"""
 * SUMMARY: This script reads CSV files from a specific Google Drive folder mounted on Google Colab.

the Folder ID is
https://drive.google.com/drive/folders/1Fq72TFN5j4Wqv0deGyXjAXRNhSCORH4s?usp=sharing

THe Folder name is Shopping CSV AmZUS

 * It filters files based on their last modification date and processes them using Pandas.
 * Specifically, it selects rows where Column B is not blank and deduplicates rows based on Column D.

 * Specifically, it selects rows where Column B is not blank and deduplicates rows based on Column D.
"""


# Folder ID from Google Drive
folder_id = '1Fq72TFN5j4Wqv0deGyXjAXRNhSCORH4s'

# Time cutoff for file modification (24 hours)
time_cutoff = datetime.utcnow() - timedelta(days=1)

# List all CSV files in the folder modified within the last 1 day
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

for file in file_list:
    if file['modifiedDate'] > time_cutoff.isoformat():
        # Download and read the CSV file into a Pandas DataFrame
        downloaded = drive.CreateFile({'id': file['id']})
        csv_data = io.StringIO(downloaded.GetContentString())
        df = pd.read_csv(csv_data)

        # Filter rows where the 2nd column (Column B) is not blank
        filtered_df = df[df.iloc[:, 1].notna()]

        # Deduplicate rows based on the 4th column (Column D)
        df_deduplicated = filtered_df.drop_duplicates(subset=df.columns[3])

        # Output or further processing
        print(f'Processed data from {file["title"]}')
        print(df_deduplicated)


Processed data from shopping_imgurl-xml_ddg_shopping_with_imgurl5-xml_ddg_shopping_with_imgurl.csv
                                                  title           price  \
0                      Fox Proframe Helmet 2023 m BLACK  $269.95$284.95   
1                Fox Proframe Helmet 2023 S BLACK/WHITE  $269.95$284.95   
3                 Fox Proframe Helmet 2023 L EUCALYPTUS  $269.95$284.95   
4                Fox Proframe Helmet 2023 m OLIVE GREEN  $269.95$284.95   
5     Fox Racing 2022 V3 RS Helmet With MIPS, Carbon...  $373.99$549.95   
...                                                 ...             ...   
1425  Trinx Latte Love By Sue Schlabach - Unframed G...         $174.99   
1426  Farmhouse Latte | Large Solid-Faced Canvas Wal...   $69.99$139.99   
1427  Wayfair Love You A Latte By Mary Urban - Textu...         $146.99   
1428  Americanflat - Mucha Latte By Anderson Design ...          $73.99   
1429  Pumpkin Spice Latte | Large Fine Art Print, Si...  $172.49$344.99   



In [59]:
df_deduplicated

,title,price,imgurl,link,date
0,Blue Merry & Bright Swaddle & Hat Set OSFA By ...,$36.00,https://external-content.duckduckgo.com/iu/?u=...,about:blank,"Friday, 20 October 2023 3:05:23 pm"
2,Baby Powder Micro 3000 Air Freshener Refill,$6.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.airdelights.com/401253?network=s&d...,"Friday, 20 October 2023 3:05:42 pm"
3,Graco Premium Foam Standard Crib & Toddler Bed...,$89.99$119.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.wayfair.com/Graco--Graco-Premium-F...,"Friday, 20 October 2023 3:05:55 pm"
4,Bright Brown Baby Pack,$21.99$26.97,https://external-content.duckduckgo.com/iu/?u=...,https://shop.scholastic.com/parent-ecommerce/b...,"Friday, 20 October 2023 3:06:04 pm"
5,Girls' & Boys' Lemonade In White Baby Zip Slee...,$24.00$40.00,https://external-content.duckduckgo.com/iu/?u=...,https://www.hannaandersson.com/pajamas-baby/60...,"Friday, 20 October 2023 3:06:13 pm"
...,...,...,...,...,...
677,Advantage Multi For Cats Over 10Lbs 6 Doses,$59.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.budgetpetcare.com/advantage-multi-...,"Friday, 20 October 2023 6:28:41 pm"
680,"Pet Hair Remover For Cars, Rubber Brush To Rem...",$14.50,https://external-content.duckduckgo.com/iu/?u=...,https://www.ebay.com/itm/314658595681?chn=ps&n...,"Friday, 20 October 2023 6:29:10 pm"
681,Atopica For Cats 100Mg/Ml - 5 Ml,$49.15,https://external-content.duckduckgo.com/iu/?u=...,https://www.allivet.com/product/atopica-for-ca...,"Friday, 20 October 2023 6:29:24 pm"
682,Blood Sugar Gold - For Cat Blood Sugar Support...,$41.95,https://external-content.duckduckgo.com/iu/?u=...,https://petwellbeing.com/products/cat-diabetes...,"Friday, 20 October 2023 6:29:36 pm"


In [60]:
# @title Normalize the Price Format


"""
 * SUMMARY: This script cleans the data in Column B based on specific rules.
"""

# Assuming df_deduplicated is your DataFrame
# Extract Column B (assuming it's the 2nd column)
column_b = df_deduplicated.iloc[:, 1]

# Clean the data
def clean_cell(cell):
    cell_str = str(cell)

    # If 'gbp' is in the cell, return 0
    if 'gbp' in cell_str.lower():
        return '0'

    # Remove commas
    cell_str = cell_str.replace(',', '')

    # If multiple dollar amounts, keep the first one
    dollar_amounts = re.findall(r'\$\d+\.\d{2}', cell_str)
    if dollar_amounts:
        return dollar_amounts[0]

    # If no dollar sign but is a number, add a dollar sign
    if re.match(r'^\d+(\.\d{2})?$', cell_str):
        return f"${cell_str}"

    return cell_str  # return original if none of the above conditions are met

cleaned_column_b = column_b.apply(clean_cell)

# Update the DataFrame
df_deduplicated.iloc[:, 1] = cleaned_column_b

print("Column B has been updated.")
print(df_deduplicated.head())


Column B has been updated.
                                               title   price  \
0  Blue Merry & Bright Swaddle & Hat Set OSFA By ...  $36.00   
2        Baby Powder Micro 3000 Air Freshener Refill   $6.99   
3  Graco Premium Foam Standard Crib & Toddler Bed...  $89.99   
4                             Bright Brown Baby Pack  $21.99   
5  Girls' & Boys' Lemonade In White Baby Zip Slee...  $24.00   

                                              imgurl  \
0  https://external-content.duckduckgo.com/iu/?u=...   
2  https://external-content.duckduckgo.com/iu/?u=...   
3  https://external-content.duckduckgo.com/iu/?u=...   
4  https://external-content.duckduckgo.com/iu/?u=...   
5  https://external-content.duckduckgo.com/iu/?u=...   

                                                link  \
0                                        about:blank   
2  https://www.airdelights.com/401253?network=s&d...   
3  https://www.wayfair.com/Graco--Graco-Premium-F...   
4  https://shop.scholastic.

<ipython-input-60-10a40e9046b2>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated.iloc[:, 1] = cleaned_column_b


In [61]:
df_deduplicated

,title,price,imgurl,link,date
0,Blue Merry & Bright Swaddle & Hat Set OSFA By ...,$36.00,https://external-content.duckduckgo.com/iu/?u=...,about:blank,"Friday, 20 October 2023 3:05:23 pm"
2,Baby Powder Micro 3000 Air Freshener Refill,$6.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.airdelights.com/401253?network=s&d...,"Friday, 20 October 2023 3:05:42 pm"
3,Graco Premium Foam Standard Crib & Toddler Bed...,$89.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.wayfair.com/Graco--Graco-Premium-F...,"Friday, 20 October 2023 3:05:55 pm"
4,Bright Brown Baby Pack,$21.99,https://external-content.duckduckgo.com/iu/?u=...,https://shop.scholastic.com/parent-ecommerce/b...,"Friday, 20 October 2023 3:06:04 pm"
5,Girls' & Boys' Lemonade In White Baby Zip Slee...,$24.00,https://external-content.duckduckgo.com/iu/?u=...,https://www.hannaandersson.com/pajamas-baby/60...,"Friday, 20 October 2023 3:06:13 pm"
...,...,...,...,...,...
677,Advantage Multi For Cats Over 10Lbs 6 Doses,$59.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.budgetpetcare.com/advantage-multi-...,"Friday, 20 October 2023 6:28:41 pm"
680,"Pet Hair Remover For Cars, Rubber Brush To Rem...",$14.50,https://external-content.duckduckgo.com/iu/?u=...,https://www.ebay.com/itm/314658595681?chn=ps&n...,"Friday, 20 October 2023 6:29:10 pm"
681,Atopica For Cats 100Mg/Ml - 5 Ml,$49.15,https://external-content.duckduckgo.com/iu/?u=...,https://www.allivet.com/product/atopica-for-ca...,"Friday, 20 October 2023 6:29:24 pm"
682,Blood Sugar Gold - For Cat Blood Sugar Support...,$41.95,https://external-content.duckduckgo.com/iu/?u=...,https://petwellbeing.com/products/cat-diabetes...,"Friday, 20 October 2023 6:29:36 pm"


In [62]:
# @title Link Cleaning


"""
 * SUMMARY: This script first cleans Bing URLs in Column D and then keeps only the part of any URL before the '?'.
 * It skips blank cells or cells that don't start with 'http'.
"""

# Assuming df_deduplicated is your DataFrame
# Extract Column D (assuming it's the 4th column)
column_d = df_deduplicated.iloc[:, 3]

# Clean the data
def clean_url(url):
    if pd.isna(url) or not url.startswith('http'):
        return url  # Skip if blank or not a URL

    parsed_url = urlparse(url)

    # Step 1: Check for Bing pattern and extract the actual URL
    if 'www.bing.com' in parsed_url.netloc:
        query_params = parse_qs(parsed_url.query)
        if 'url' in query_params:
            url = query_params['url'][0]
            parsed_url = urlparse(url)

    # Step 2: Keep only the part before the '?'
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
    return base_url

cleaned_column_d = column_d.apply(clean_url)

# Update the DataFrame
df_deduplicated.iloc[:, 3] = cleaned_column_d

print("Column D has been updated.")



Column D has been updated.


<ipython-input-62-6598a9bddae0>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated.iloc[:, 3] = cleaned_column_d


In [63]:
df_deduplicated

,title,price,imgurl,link,date
0,Blue Merry & Bright Swaddle & Hat Set OSFA By ...,$36.00,https://external-content.duckduckgo.com/iu/?u=...,about:blank,"Friday, 20 October 2023 3:05:23 pm"
2,Baby Powder Micro 3000 Air Freshener Refill,$6.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.airdelights.com/401253,"Friday, 20 October 2023 3:05:42 pm"
3,Graco Premium Foam Standard Crib & Toddler Bed...,$89.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.wayfair.com/Graco--Graco-Premium-F...,"Friday, 20 October 2023 3:05:55 pm"
4,Bright Brown Baby Pack,$21.99,https://external-content.duckduckgo.com/iu/?u=...,https://shop.scholastic.com/parent-ecommerce/b...,"Friday, 20 October 2023 3:06:04 pm"
5,Girls' & Boys' Lemonade In White Baby Zip Slee...,$24.00,https://external-content.duckduckgo.com/iu/?u=...,https://www.hannaandersson.com/pajamas-baby/60...,"Friday, 20 October 2023 3:06:13 pm"
...,...,...,...,...,...
677,Advantage Multi For Cats Over 10Lbs 6 Doses,$59.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.budgetpetcare.com/advantage-multi-...,"Friday, 20 October 2023 6:28:41 pm"
680,"Pet Hair Remover For Cars, Rubber Brush To Rem...",$14.50,https://external-content.duckduckgo.com/iu/?u=...,https://www.ebay.com/itm/314658595681,"Friday, 20 October 2023 6:29:10 pm"
681,Atopica For Cats 100Mg/Ml - 5 Ml,$49.15,https://external-content.duckduckgo.com/iu/?u=...,https://www.allivet.com/product/atopica-for-ca...,"Friday, 20 October 2023 6:29:24 pm"
682,Blood Sugar Gold - For Cat Blood Sugar Support...,$41.95,https://external-content.duckduckgo.com/iu/?u=...,https://petwellbeing.com/products/cat-diabetes,"Friday, 20 October 2023 6:29:36 pm"



*follow* the pattern  https://duckduckgo.com/?q=Infant+Baby+Toddler+Safety+Helmet+Kids+Walking+Crawling+Head+Protection+site%3Aamazon.com&va=o&t=hx&ia=web

In [64]:


"""
 * SUMMARY: This script generates DuckDuckGo search URLs based on product titles in Column A.

Remove all commas
Convert to lowercase
Remove all non-alphanumeric characters
Trim spaces
use the first 10 words of each title and

generate a URL with the pattern:

https://duckduckgo.com/?q= words from the title +site%3Aamazon.com&va=o&t=hx&ia=web

 * The generated URLs are placed in a new column titled 'ddg_amazon'.
 * Finally, the DataFrame is saved to a CSV file.

 * SUMMARY: This script generates DuckDuckGo search URLs based on product titles in Column A.
 * It normalizes the titles by removing commas, converting to lowercase, removing non-alphanumeric characters, and trimming spaces.
 * It then uses the first 10 words of each title to generate a URL.

 * The generated URLs are placed in a new column titled 'ddg_amazon'.
 * Finally, the DataFrame is saved to a CSV file.
"""

# Assuming df_deduplicated is your DataFrame
# Extract Column A (assuming it's the 1st column)
column_a = df_deduplicated.iloc[:, 0]

# Generate DuckDuckGo URL
def generate_ddg_url(title):
    if pd.isna(title):
        return None  # Skip if blank

    # Normalize the title
    title = title.replace(',', '').lower()
    title = re.sub(r'[^a-zA-Z0-9\s]', '', title).strip()

    # Take the first 10 words
    words = title.split()[:10]

    # Join the words and replace spaces with '+'
    query = "+".join(words)





    # Create DuckDuckGo URL following the specified pattern
    ddg_url = f"https://duckduckgo.com/?q={query}+site%3Aamazon.com&va=o&t=hx&ia=web"

    return ddg_url

# Generate DuckDuckGo URLs for Column A
ddg_amazon = column_a.apply(generate_ddg_url)

# Update the DataFrame
df_deduplicated['ddg_amazon'] = ddg_amazon

# Save to CSV
save_path = '/content/drive/MyDrive/Amazon/amz_arbitrage/amz_arbitrage_sources/AMZUS Folder/shopping_csv_AMZUS/processed_imgurl_csv/ddg_shopping_with_imgurl_5_columns.csv'

# Check if the CSV file already exists
if os.path.exists(save_path):
    # Load existing data
    existing_data = pd.read_csv(save_path)

    # Append new data
    combined_data = pd.concat([existing_data, df_deduplicated], ignore_index=True)

    # Remove duplicates
    combined_data = combined_data.drop_duplicates()

    # Save the combined DataFrame to the CSV file
    combined_data.to_csv(save_path, index=False)
else:
    # Save the DataFrame to a new CSV file
    df_deduplicated.to_csv(save_path, index=False)

print(f"The new column 'ddg_amazon' has been created with normalized DuckDuckGo URLs, and the DataFrame has been saved to {save_path}.")


The new column 'ddg_amazon' has been created with normalized DuckDuckGo URLs, and the DataFrame has been saved to /content/drive/MyDrive/Amazon/amz_arbitrage/amz_arbitrage_sources/AMZUS Folder/shopping_csv_AMZUS/processed_imgurl_csv/ddg_shopping_with_imgurl_5_columns.csv.


<ipython-input-64-8c23f7d1042d>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated['ddg_amazon'] = ddg_amazon


In [65]:
df_deduplicated

,title,price,imgurl,link,date,ddg_amazon
0,Blue Merry & Bright Swaddle & Hat Set OSFA By ...,$36.00,https://external-content.duckduckgo.com/iu/?u=...,about:blank,"Friday, 20 October 2023 3:05:23 pm",https://duckduckgo.com/?q=blue+merry+bright+sw...
2,Baby Powder Micro 3000 Air Freshener Refill,$6.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.airdelights.com/401253,"Friday, 20 October 2023 3:05:42 pm",https://duckduckgo.com/?q=baby+powder+micro+30...
3,Graco Premium Foam Standard Crib & Toddler Bed...,$89.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.wayfair.com/Graco--Graco-Premium-F...,"Friday, 20 October 2023 3:05:55 pm",https://duckduckgo.com/?q=graco+premium+foam+s...
4,Bright Brown Baby Pack,$21.99,https://external-content.duckduckgo.com/iu/?u=...,https://shop.scholastic.com/parent-ecommerce/b...,"Friday, 20 October 2023 3:06:04 pm",https://duckduckgo.com/?q=bright+brown+baby+pa...
5,Girls' & Boys' Lemonade In White Baby Zip Slee...,$24.00,https://external-content.duckduckgo.com/iu/?u=...,https://www.hannaandersson.com/pajamas-baby/60...,"Friday, 20 October 2023 3:06:13 pm",https://duckduckgo.com/?q=girls+boys+lemonade+...
...,...,...,...,...,...,...
677,Advantage Multi For Cats Over 10Lbs 6 Doses,$59.99,https://external-content.duckduckgo.com/iu/?u=...,https://www.budgetpetcare.com/advantage-multi-...,"Friday, 20 October 2023 6:28:41 pm",https://duckduckgo.com/?q=advantage+multi+for+...
680,"Pet Hair Remover For Cars, Rubber Brush To Rem...",$14.50,https://external-content.duckduckgo.com/iu/?u=...,https://www.ebay.com/itm/314658595681,"Friday, 20 October 2023 6:29:10 pm",https://duckduckgo.com/?q=pet+hair+remover+for...
681,Atopica For Cats 100Mg/Ml - 5 Ml,$49.15,https://external-content.duckduckgo.com/iu/?u=...,https://www.allivet.com/product/atopica-for-ca...,"Friday, 20 October 2023 6:29:24 pm",https://duckduckgo.com/?q=atopica+for+cats+100...
682,Blood Sugar Gold - For Cat Blood Sugar Support...,$41.95,https://external-content.duckduckgo.com/iu/?u=...,https://petwellbeing.com/products/cat-diabetes,"Friday, 20 October 2023 6:29:36 pm",https://duckduckgo.com/?q=blood+sugar+gold+for...


In [66]:
# @title Upload ASIN Harvy Data


"""
 * SUMMARY: This script reads all CSV files from a specified folder into Pandas DataFrames, renames the third column to 'ddg_amazon', normalizes the URLs, and displays the first few rows of each.
"""

# Initialize Google Drive API client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Function to normalize DuckDuckGo URLs
def normalize_ddg_url(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)
    if 'q' in query_params:
        query_params['q'] = [query_params['q'][0].replace('%2B', '+')]
    normalized_url = urlunparse(
        (parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, urlencode(query_params, doseq=True), parsed_url.fragment)
    )
    return normalized_url

# Folder ID
folder_id = '1f05bN7eLleHD2iRf5zLRLUjxcFVplnka'

# Query to list files
query = f"'{folder_id}' in parents and trashed=false"
file_list = drive.ListFile({'q': query}).GetList()

# Loop through each file to read and process
for file in file_list:
    if file['title'].endswith('.csv'):
        print(f"Processing file: {file['title']}")
        file_id = file['id']
        downloaded = drive.CreateFile({'id': file_id})
        downloaded.GetContentFile('temp.csv')
        reverse_asin = pd.read_csv('temp.csv')
        reverse_asin.columns.values[2] = 'ddg_amazon'
        reverse_asin['ddg_amazon'] = reverse_asin['ddg_amazon'].apply(normalize_ddg_url)
        print(reverse_asin.head())




Processing file: ddg_reverse_lookup-xml_reverse_ASIN_lookup_ddg_3_columns-3-xml_reverse_ASIN_lookup_ddg_3_columns.csv
                                               desc1  \
0  amazon.com Report Ad Shop head holder for baby...   
1  amazon.com Report Ad Shop swaddle and hat set ...   
2  airdelights.com Report Ad Micro 3000 Air Neutr...   
3  walmart.com Report Ad Crib and Baby - Crib & T...   
4  amazon.com Report Ad Shop bag pack for baby - ...   

                                               desc2  \
0  amazon.com Report Ad Shop head holder for baby...   
1  amazon.com Report Ad Shop swaddle and hat set ...   
2  airdelights.com Report Ad Micro 3000 Air Neutr...   
3  walmart.com Report Ad Crib and Baby - Crib & T...   
4  amazon.com Report Ad Shop bag pack for baby - ...   

                                          ddg_amazon  
0  https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...  
1  https://duckduckgo.com/?q=blue+merry+bright+sw...  
2  https://duckduckgo.com/?q=baby+powder+mi

In [67]:
reverse_asin

,desc1,desc2,ddg_amazon
0,amazon.com Report Ad Shop head holder for baby...,amazon.com Report Ad Shop head holder for baby...,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...
1,amazon.com Report Ad Shop head holder for baby...,amazon.com Report Ad Shop head holder for baby...,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...
2,temu.com Report Ad Temu Baby Head Protector - ...,temu.com Report Ad Temu Baby Head Protector - ...,https://duckduckgo.com/?q=Baby+Safety+Helmet+H...
3,target.com Report Ad Sports Equipment at Targe...,target.com Report Ad Sports Equipment at Targe...,https://duckduckgo.com/?q=Baby+Helmet+For+Craw...
4,temu.com Report Ad Safety Helmet for Baby - Ad...,temu.com Report Ad Safety Helmet for Baby - Ad...,https://duckduckgo.com/?q=Baby+Safety+Helmet+I...
...,...,...,...
311,foxracing.com Report Ad Fox Racing® - Official...,foxracing.com Report Ad Fox Racing® - Official...,https://duckduckgo.com/?q=Fox+Apparel+%7C+PROF...
312,foxracing.com Report Ad Fox Speedframe Pro - O...,foxracing.com Report Ad Fox Speedframe Pro - O...,https://duckduckgo.com/?q=Fox+Racing+Speedfram...
313,foxracing.com Report Ad Fox Speedframe MTB Hel...,foxracing.com Report Ad Fox Speedframe MTB Hel...,https://duckduckgo.com/?q=Fox+Racing+Speedfram...
314,foxracing.com Report Ad Shop Fox Racing Speedf...,foxracing.com Report Ad Shop Fox Racing Speedf...,https://duckduckgo.com/?q=Fox+Racing+Speedfram...


In [68]:
import re

# Define a function to find ASINs in text
def find_asins_in_text(text):
    asin_pattern = r'[A-Z0-9]{10}'
    asins = re.findall(asin_pattern, text)
    return ', '.join(asins) if asins else None

# Apply the function to the first two columns and create a new 'ASIN' column
reverse_asin['ASIN'] = reverse_asin.apply(lambda row: find_asins_in_text(row[0] + row[1]), axis=1)

# Display the updated DataFrame
print(reverse_asin.head())


                                               desc1  \
0  amazon.com Report Ad Shop head holder for baby...   
1  amazon.com Report Ad Shop head holder for baby...   
2  temu.com Report Ad Temu Baby Head Protector - ...   
3  target.com Report Ad Sports Equipment at Targe...   
4  temu.com Report Ad Safety Helmet for Baby - Ad...   

                                               desc2  \
0  amazon.com Report Ad Shop head holder for baby...   
1  amazon.com Report Ad Shop head holder for baby...   
2  temu.com Report Ad Temu Baby Head Protector - ...   
3  target.com Report Ad Sports Equipment at Targe...   
4  temu.com Report Ad Safety Helmet for Baby - Ad...   

                                          ddg_amazon  \
0  https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...   
1  https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...   
2  https://duckduckgo.com/?q=Baby+Safety+Helmet+H...   
3  https://duckduckgo.com/?q=Baby+Helmet+For+Craw...   
4  https://duckduckgo.com/?q=Baby+Safety+Helme

In [69]:
"""
 * SUMMARY: This script processes a Pandas DataFrame by splitting and exploding the 'ASIN' column,
 * optionally removing specified columns, and removing duplicate rows.
"""

# Assuming reverse_asin is your DataFrame
# reverse_asin = pd.read_csv(csv_file_path)

# Split the 'ASIN' column values into a list
reverse_asin['ASIN'] = reverse_asin['ASIN'].str.split(', ')

# Explode the 'ASIN' column to create a row for each ASIN
reverse_asin = reverse_asin.explode('ASIN')

# Check if the columns ['desc1', 'desc2'] exist
if 'desc1' in reverse_asin.columns and 'desc2' in reverse_asin.columns:
    # Remove the first two columns (desc1 and desc2)
    reverse_asin = reverse_asin.drop(['desc1', 'desc2'], axis=1)
else:
    print("Columns 'desc1' and/or 'desc2' do not exist.")

reverse_asin


,ddg_amazon,ASIN
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B07FM2ZVYM
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B012WGB3Q4
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B08BYK5W7S
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B09WHXSSYR
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B09NVRSB45
...,...,...
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,B08W21M77M
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,SPEEDFRAME
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,SPEEDFRAME
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,B0B649V38K


In [70]:
import pandas as pd
import os

"""
 * SUMMARY: This script saves a processed Pandas DataFrame to a CSV file.
 * If the CSV file already exists and contains data, the script appends new rows and removes duplicates.
"""

# Assuming reverse_asin is your DataFrame
# reverse_asin = pd.read_csv(csv_file_path)

# Define the path where you want to save the CSV file
csv_path = '/content/drive/MyDrive/Amazon/amz_arbitrage/amz_arbitrage_sources/AMZUS Folder/ddg_reverse_lookup_3_columns/processed_ASIN_lookup_2_cols/ddg_asin_2_cols.csv'

# Check if the CSV file already exists
if os.path.exists(csv_path):
    # Load existing data
    existing_data = pd.read_csv(csv_path)

    # Append new data
    combined_data = pd.concat([existing_data, reverse_asin], ignore_index=True)

    # Remove duplicates
    combined_data = combined_data.drop_duplicates()

    # Save the combined DataFrame to the CSV file
    combined_data.to_csv(csv_path, index=False)
else:
    # Save the 'reverse_asin' DataFrame to a new CSV file
    reverse_asin.to_csv(csv_path, index=False)

# The DataFrame is now saved to the specified location
reverse_asin


,ddg_amazon,ASIN
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B07FM2ZVYM
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B012WGB3Q4
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B08BYK5W7S
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B09WHXSSYR
0,https://duckduckgo.com/?q=Baby+Toddler+Head+Pr...,B09NVRSB45
...,...,...
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,B08W21M77M
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,SPEEDFRAME
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,SPEEDFRAME
315,https://duckduckgo.com/?q=Fox+Racing+Mainframe...,B0B649V38K


In [71]:
import pandas as pd

"""

df1 is reverse_asin
df2 is df_deduplicated
Common column is 'ddg_amazon'
common_column is 'ddg_amazon'


update the column by following the example.

https://duckduckgo.com/?q=Baby+Toddler+Head+Protector%2C+Upgrade+Infant+Safety+Helmet%2C+Breathable+Head++site:amazon.com&va=o&t=hx&ia=web
should be
baby toddler head protector upgrade infant safety helmet breathable head



then
# Merge the DataFrames on the common column using a left merge
The merge type is 'left', keeping all rows from reverse_asin and matching rows from df_deduplicated.



import pandas as pd
from urllib.parse import urlparse, parse_qs

"""


import re

# Function to extract and normalize the query part of a DuckDuckGo URL
def extract_query_from_ddg_url(url):
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)

    if 'q' in query_params:
        query = query_params['q'][0]
        # Remove the site:amazon.com part
        query = query.replace('site:amazon.com', '')
        # Replace '+' and '%2B' with space
        query = query.replace('+', ' ').replace('%2B', ' ')
        # Remove all commas
        query = query.replace(',', '')
        # Convert to lowercase
        query = query.lower()

        # Remove all non-alphanumeric characters
        query = re.sub(r'[^a-zA-Z0-9\s]', '', query)

        # Trim spaces
        query = query.strip()

        return query
    return None


# Assuming reverse_asin and df_deduplicated are your DataFrames
# Common column is 'ddg_amazon'
common_column = 'ddg_amazon'

# Update the 'ddg_amazon' column in both DataFrames
reverse_asin[common_column] = reverse_asin[common_column].apply(extract_query_from_ddg_url)
df_deduplicated[common_column] = df_deduplicated[common_column].apply(extract_query_from_ddg_url)

# Merge the DataFrames on the common column using inner merge
merged_df = pd.merge(reverse_asin, df_deduplicated, on=common_column, how='inner')



<ipython-input-71-0d953f56c1ac>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deduplicated[common_column] = df_deduplicated[common_column].apply(extract_query_from_ddg_url)


In [86]:
merged_df

,ddg_amazon,ASIN,title,price,imgurl,link,date,match_code


In [72]:
import pandas as pd

"""
 * SUMMARY: This script joins the cells in column B with the cells in column F using an asterisk (*) as the separator.
 * The result is stored in a new column named 'match_code'.
"""

# Assuming merged_df is your DataFrame
# You can read it from a CSV file if needed: merged_df = pd.read_csv('your_file.csv')

# Join the cells in column B and F with an asterisk (*) and update column H ('match_code')
merged_df['match_code'] = merged_df['ASIN'].astype(str) + '*' + merged_df['link'].astype(str)

# Save the updated DataFrame back to a CSV file if needed
# merged_df.to_csv('your_updated_file.csv', index=False)


In [73]:
""""
 * SUMMARY: This script removes duplicate rows in the DataFrame based on the 'match_code' column.
"""
# Remove duplicate rows based on the 'match_code' column

merged_df.drop_duplicates(subset=['match_code'], inplace=True)

# Display the DataFrame to check that duplicates have been removed


,ddg_amazon,ASIN,title,price,imgurl,link,date,match_code


In [74]:
merged_df

,ddg_amazon,ASIN,title,price,imgurl,link,date,match_code


In [75]:
# @title Download match_table to CSV

import pandas as pd
import os

"""
 * SUMMARY: This script saves a merged DataFrame to a CSV file.
 * If the CSV file already exists and contains data, the script appends new rows and removes duplicates.
 * Then removes any rows where the column match_code is blank.
"""

# Assuming merged_df is your DataFrame
# merged_df = pd.merge(reverse_asin, df_deduplicated, on=common_column, how='left')

# Define the path where you want to save the CSV file
save_path = '/content/drive/MyDrive/Amazon/amz_arbitrage/amz_arbitrage_sources/AMZUS Folder/shopping_csv_AMZUS/processed_imgurl_csv/match_table.csv'

try:
    # Check if the CSV file already exists
    if os.path.exists(save_path):
        # Load existing data
        existing_data = pd.read_csv(save_path)

        # Append new data
        combined_data = pd.concat([existing_data, merged_df], ignore_index=True)

        # Remove duplicates
        combined_data = combined_data.drop_duplicates()

        # Remove rows where match_code is blank
        combined_data = combined_data[combined_data['match_code'].notna()]

        # Save the combined DataFrame to the CSV file
        combined_data.to_csv(save_path, index=False)
    else:
        # Remove rows where match_code is blank in merged_df
        merged_df = merged_df[merged_df['match_code'].notna()]

        # Save the merged_df DataFrame to a new CSV file
        merged_df.to_csv(save_path, index=False)

    print(f"The DataFrame has been saved to {save_path}.")

except Exception as e:
    print(f"Error: {e}")



The DataFrame has been saved to /content/drive/MyDrive/Amazon/amz_arbitrage/amz_arbitrage_sources/AMZUS Folder/shopping_csv_AMZUS/processed_imgurl_csv/match_table.csv.


In [76]:
# @title Find Keepa Data


""""
Find all CSV files modified in the last 2 days
in the folders with ID

folder_ids = ['1-BLjdr1J7OrD5A1SDFnl8pHumXjEYZ7H', '1-8HkYAOZCowdEuia_2fumIcpt3xCqH0k']

where the file starts with Keepa
where the first column contains cell with the string "com"


Upload the CSV files to
df with the name keepa_data

remove duplicates
"""
# Folder IDs for downloading Keepa files
folder_ids = ['1-BLjdr1J7OrD5A1SDFnl8pHumXjEYZ7H', '1-8HkYAOZCowdEuia_2fumIcpt3xCqH0k', '1FBJxq2F3XVfMyez4EWLSIact3mFVpxG1']

# Get the current date and time in UTC
now = datetime.utcnow()

# Initialize an empty DataFrame to hold all the Keepa data
keepa_data = pd.DataFrame()

# Loop through each folder ID to list files
for folder_id in folder_ids:
    print(f"Listing files for folder ID: {folder_id}")

    # Query to list files that are not in the trash
    query = f"'{folder_id}' in parents and trashed=false"
    file_list = drive.ListFile({'q': query}).GetList()

    # List files that are CSV, modified within the last 2 days, and start with "Keepa"
    for file in file_list:
        if file['title'].startswith('Keepa') and file['title'].endswith('.csv'):
            modified_time = datetime.strptime(file['modifiedDate'], '%Y-%m-%dT%H:%M:%S.%fZ')
            if now - modified_time <= timedelta(days=2):
                print(f"Title: {file['title']}, ID: {file['id']}, Modified Date: {file['modifiedDate']}")

                # Download the file and read it into a DataFrame
                file_id = file['id']
                downloaded = drive.CreateFile({'id': file_id})
                downloaded.GetContentFile('temp.csv')
                temp_df = pd.read_csv('temp.csv')

                # Append to the main DataFrame
                keepa_data = pd.concat([keepa_data, temp_df], ignore_index=True)

    print("------")

# Remove duplicates from the main DataFrame
keepa_data = keepa_data.drop_duplicates()

# Upload the DataFrame to Google Drive
upload_folder_id = '1FBJxq2F3XVfMyez4EWLSIact3mFVpxG1'  # Replace with your folder ID
keepa_data.to_csv('keepa_AMZUS.csv', index=False)
upload_file = drive.CreateFile({'title': 'keepa_AMZUS.csv', 'parents': [{'id': upload_folder_id}]})
upload_file.SetContentFile('keepa_AMZUS.csv')
upload_file.Upload()

print(f"The DataFrame has been saved to Google Drive folder with ID {upload_folder_id}.")


Listing files for folder ID: 1-BLjdr1J7OrD5A1SDFnl8pHumXjEYZ7H
------
Listing files for folder ID: 1-8HkYAOZCowdEuia_2fumIcpt3xCqH0k
Title: KeepaExport-2023-10-21-ProductViewer.csv, ID: 1LdXCFcLvf41OHf1iuQL1L1FR7VfbJQy5, Modified Date: 2023-10-20T23:41:07.813Z
Title: KeepaExport-2023-10-20-ProductFinder (1).csv, ID: 1LaOPYdi1KWNnUBvU4cQmbmSDHrwPQEle, Modified Date: 2023-10-20T15:24:08.468Z
Title: KeepaExport-2023-10-20-ProductFinder.csv, ID: 1L_3Jzhtoxeo-d0JD9t1cX_iHIpUWMrDr, Modified Date: 2023-10-20T15:22:26.944Z
Title: KeepaExport-2023-10-20-ProductViewer (1).csv, ID: 1LBPInu4mQ-DFp61hwGzCALccYXGpRcJs, Modified Date: 2023-10-20T05:03:54.035Z
Title: KeepaExport-2023-10-20-ProductViewer.csv, ID: 1L9IuD4LlsC02QQ3dqDfzd1WU4BG-nzKU, Modified Date: 2023-10-20T03:40:51.446Z
------
Listing files for folder ID: 1FBJxq2F3XVfMyez4EWLSIact3mFVpxG1
------
The DataFrame has been saved to Google Drive folder with ID 1FBJxq2F3XVfMyez4EWLSIact3mFVpxG1.


In [77]:
keepa_data

,Locale,Image,Title,Sales Rank: 30 days avg.,Bought in past month,Buy Box 🚚: Current,Buy Box 🚚: 90 days avg.,FBA Pick&Pack Fee,Referral Fee %,New Offer Count: 90 days avg.,URL: Amazon,ASIN,Product Codes: EAN,Product Codes: UPC,Product Codes: PartNumber,Brand
0,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Bike-Helmets Speedframe Pro Mountai...,NaN,NaN,NaN,NaN,-,NaN,NaN,https://www.amazon.com/dp/B0BQQ3SD6X,B0BQQ3SD6X,NaN,NaN,NaN,Fox Racing
1,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Speedframe Pro Helmet,174951.0,NaN,NaN,NaN,-,NaN,NaN,https://www.amazon.com/dp/B095RSR46H,B095RSR46H,NaN,NaN,NaN,Fox Racing
2,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Speedframe Mountain Bike Helmet,25468.0,NaN,NaN,NaN,-,NaN,NaN,https://www.amazon.com/dp/B0B8N55YCC,B0B8N55YCC,NaN,NaN,NaN,Fox Racing
3,com,https://images-na.ssl-images-amazon.com/images...,"Fox Racing Proframe Mountain Bike Helmet, BLOC...",682707.0,NaN,$ 227.96,$ 257.95,$ 6.60,15 %,6.0,https://www.amazon.com/dp/B0988SZ37T,B0988SZ37T,0191972616440,191972616440,29366-001,Fox Racing
4,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Speedframe Pro Mountain Bike Helmet...,19984.0,NaN,$ 189.95,$ 188.55,$ 9.11,15 %,12.0,https://www.amazon.com/dp/B0988X9SPG,B0988X9SPG,0191972614163,191972614163,29341-001,Fox Racing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10486,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Speedframe MIPS Pro Helmet Repeater...,172332.0,NaN,$ 94.95,$ 96.09,$ 9.11,15 %,3.0,https://www.amazon.com/dp/B08W1P9FP8,B08W1P9FP8,0191972520259,191972520259,27503,Fox Racing
10497,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing V3 RS Helmet Visor,4372554.0,NaN,$ 39.95,$ 39.95,$ 7.83,15 %,2.0,https://www.amazon.com/dp/B0964CC16C,B0964CC16C,0191972563362,191972563362,29223-119,Fox Racing
10524,com,https://images-na.ssl-images-amazon.com/images...,"Fox Racing DROPFRAME FIT KIT, Black, OS",977473.0,NaN,$ 24.95,$ 24.95,$ 5.99,15 %,3.0,https://www.amazon.com/dp/B07SD3D7CT,B07SD3D7CT,0191972266621,191972266621,25201-001-OS,Fox Racing
10532,com,https://images-na.ssl-images-amazon.com/images...,Fox Racing Speedframe MIPS Pro Helmet Lunar/Li...,174771.0,50.0,$ 139.95,$ 120.20,$ 9.11,15 %,2.0,https://www.amazon.com/dp/B0B4ZQRK22,B0B4ZQRK22,0191972580680,191972580680,28428-097L,Fox Racing


In [78]:

"""
df1 is  merged_df
df2 is  keepa_data
the common column is ASIN

merge the two , inner  merge

export the result_df to csv
with file name   amzus_arbitrage_data.csv
in the Folder ID  https://drive.google.com/drive/folders/1G-6hDm8AXI-kEaLnVczABqJ9KhJU26Ia?usp=share_link


"""
# Perform an inner merge on the common column 'ASIN'
result_df = pd.merge(merged_df, keepa_data, on='ASIN', how='inner')

# Remove duplicates from the merged DataFrame
result_df = result_df.drop_duplicates()

# Define the folder ID and file name
folder_id = '1G-6hDm8AXI-kEaLnVczABqJ9KhJU26Ia'
file_name = 'amzus_arbitrage_data.csv'

# Search for the file in the folder
query = f"title='{file_name}' and '{folder_id}' in parents and trashed=false"
file_list = drive.ListFile({'q': query}).GetList()

# Check if the file exists
if file_list:
    # File exists, download it and read into a DataFrame
    file_id = file_list[0]['id']
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile('temp.csv')
    existing_data = pd.read_csv('temp.csv')

    # Append new data and remove duplicates
    combined_data = pd.concat([existing_data, result_df], ignore_index=True).drop_duplicates()

    # Save the combined DataFrame to the same CSV file
    combined_data.to_csv('temp.csv', index=False)
    downloaded.SetContentFile('temp.csv')
    downloaded.Upload()
else:
    # File doesn't exist, create a new one
    result_df.to_csv('temp.csv', index=False)
    uploaded = drive.CreateFile({
        'title': file_name,
        'parents': [{'id': folder_id}]
    })
    uploaded.SetContentFile('temp.csv')
    uploaded.Upload()

print(f"The DataFrame has been exported to a CSV file and uploaded to Google Drive folder with ID {folder_id}.")


The DataFrame has been exported to a CSV file and uploaded to Google Drive folder with ID 1G-6hDm8AXI-kEaLnVczABqJ9KhJU26Ia.


In [79]:
result_df

,ddg_amazon,title,price,imgurl,link,date,match_code,Locale,Image,Title,...,Buy Box 🚚: 90 days avg.,FBA Pick&Pack Fee,Referral Fee %,New Offer Count: 90 days avg.,URL: Amazon,ASIN,Product Codes: EAN,Product Codes: UPC,Product Codes: PartNumber,Brand


In [80]:
"""
 * SUMMARY
 * This script reads a DataFrame named 'result_df' with two columns 'imgurl' and 'Image',
 * uses OpenCV for feature-based image matching, and appends the matching scores
 * to a new column 'score_cv' in the DataFrame.
"""

# Import necessary libraries
import cv2
import numpy as np
import pandas as pd
import urllib.request

def get_image_from_url(url):
    try:
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)
        return image
    except:
        return None

def compute_matching_score(image1, image2):
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)

    index_params = dict(algorithm=0, trees=5)  # FLANN_INDEX_KDTREE = 0
    search_params = dict()
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]

    return len(good_matches)

# Assume result_df exists or is loaded from a file
# result_df = pd.read_csv("your_file.csv")

matching_results = []

for index, row in result_df.iterrows():
    url1, url2 = row['imgurl'], row['Image']
    image1 = get_image_from_url(url1)
    image2 = get_image_from_url(url2)

    if image1 is not None and image2 is not None:
        score = compute_matching_score(image1, image2)
        matching_results.append(score)
    else:
        matching_results.append('Error')

result_df['score_cv'] = matching_results


In [81]:
"""
 * SUMMARY
 * This script reads a DataFrame named 'result_df' with two columns 'ddg_amazon' and 'Title',
 * calculates the similarity between each pair of product titles using Cosine Similarity,
 * and appends the similarity scores to a new column 'score_cos' in the DataFrame.
"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Function to calculate cosine similarity between two titles
def compute_cosine_similarity(title1, title2):
    vectorizer = CountVectorizer().fit_transform([title1, title2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0, 1]

# Assume result_df exists or is loaded from a file
# result_df = pd.read_csv("your_file.csv")

# Initialize list to store similarity scores
similarity_scores = []

# Loop through each row in DataFrame
for index, row in result_df.iterrows():
    title1, title2 = row['ddg_amazon'], row['Title']
    similarity_score = compute_cosine_similarity(title1, title2)
    similarity_scores.append(similarity_score)

# Add similarity scores to the DataFrame
result_df['score_cos'] = similarity_scores


In [82]:
result_df


,ddg_amazon,title,price,imgurl,link,date,match_code,Locale,Image,Title,...,Referral Fee %,New Offer Count: 90 days avg.,URL: Amazon,ASIN,Product Codes: EAN,Product Codes: UPC,Product Codes: PartNumber,Brand,score_cv,score_cos


In [83]:
"""
 * SUMMARY
 * This script exports the DataFrame 'result_df' to a CSV file in a specific Google Drive folder.
 * It removes duplicates based on the 'match_code' column, preferring newer rows.
"""


import io



folder_id = '1G-6hDm8AXI-kEaLnVczABqJ9KhJU26Ia'
local_file_path = '/content/drive/My Drive/amzus_arbitrage_data.csv'

# Check if file already exists in the folder
query = f"'{folder_id}' in parents"
file_list = drive.ListFile({'q': query}).GetList()
existing_file = next((file for file in file_list if file['title'] == 'amzus_arbitrage_data.csv'), None)

# If file exists, download and deduplicate rows
if existing_file:
    downloaded = drive.CreateFile({'id': existing_file['id']})
    content = downloaded.GetContentString()
    existing_df = pd.read_csv(io.StringIO(content))

    # Concatenate existing DataFrame with new DataFrame and remove duplicates
    combined_df = pd.concat([existing_df, result_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset='match_code', keep='last')
else:
    combined_df = result_df

# Export combined DataFrame to CSV
combined_df.to_csv(local_file_path, index=False)

# Update or upload the file to Google Drive
if existing_file:
    existing_file.SetContentFile(local_file_path)
    existing_file.Upload()
else:
    new_file = drive.CreateFile({
        'title': 'amzus_arbitrage_data.csv',
        'parents': [{'id': folder_id}]
    })
    new_file.SetContentFile(local_file_path)
    new_file.Upload()
